In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from preprocessing import clsf_tty_cd_grp, clsf_boz_cd

In [2]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('./data/보유리스크율_일반')
BASE_YYMM = '201912'

In [3]:
# 데이터 불러오기
일반_원수_경과보험료 = pd.read_excel(FILE_PATH / f'일반_원수_경과보험료_{BASE_YYMM}.xlsx', dtype={'CLG_YM': str, 'ARC_INPL_CD': str})
일반_출재_경과보험료 = pd.read_excel(FILE_PATH / f'일반_출재_경과보험료_{BASE_YYMM}.xlsx', dtype={'CLG_YM': str, 'ARC_INPL_CD': str})
일반_원수_손해액 = pd.read_excel(FILE_PATH / f'일반_원수_손해액_{BASE_YYMM}.xlsx', dtype={'CLG_YM': str, 'ARC_INPL_CD': str})
일반_출재_손해액 = pd.read_excel(FILE_PATH / f'일반_출재_손해액_{BASE_YYMM}.xlsx', dtype={'CLG_YM': str, 'ARC_INPL_CD': str})
일반_특약보종별_직전1년경과보험료 = pd.read_excel(FILE_PATH / f'일반_특약보종별_직전1년경과보험료_{BASE_YYMM}.xlsx', dtype={'RRNR_TTY_CD': str, 'ARC_INPL_CD': str})
일반_특약보종별_직전1년손해액 = pd.read_excel(FILE_PATH / f'일반_특약보종별_직전1년손해액_{BASE_YYMM}.xlsx', dtype={'RRNR_TTY_CD': str, 'ARC_INPL_CD': str})
일반_특약수수료 = pd.read_excel(FILE_PATH / f'일반_특약수수료_{202012}.xlsx', dtype={'RRNR_TTY_CD': str, 'TTY_YR': str})
일반_특약정보 = pd.read_excel(FILE_PATH / f'일반_특약정보_{BASE_YYMM}.xlsx', dtype={'RRNR_TTY_CD': str, 'TTY_YR': str})
일반_상품정보 = pd.read_excel(FILE_PATH / '일반_상품정보.xlsx', dtype={'PDC_CD': str, 'PDGR_CD': str})

In [91]:
# 데이터 전처리
## 특약정보, 특약수수료
일반_특약정보_가공 = 일반_특약정보.copy()
일반_특약정보_가공['TTY_CD_GRP'] = clsf_tty_cd_grp(일반_특약정보_가공)
일반_특약정보_가공 = 일반_특약정보_가공.drop('TTY_CD_NM', axis=1)
일반_특약수수료_가공 = 일반_특약정보_가공 \
    .merge(일반_특약수수료[['RRNR_TTY_CD', 'CMSN_ADD_RT', 'CMSN_MULT_RT', 'BSE_LSRT', 'LWT_CMSN_RT', 'TOP_CMSN_RT']], on='RRNR_TTY_CD', how='left') \
    [['TTY_YR', 'TTY_CD_GRP', 'CMSN_ADD_RT', 'CMSN_MULT_RT', 'BSE_LSRT', 'LWT_CMSN_RT', 'TOP_CMSN_RT']] \
    .loc[lambda x: ~x.CMSN_ADD_RT.isna()].drop_duplicates() \
    .sort_values(by=['TTY_CD_GRP', 'TTY_YR'])

## 경과보험료
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료.copy()
일반_특약보종별_직전1년경과보험료_가공['DMFR_DVCD'] = '01'
일반_특약보종별_직전1년경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_특약보종별_직전1년경과보험료_가공, 일반_상품정보)
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료_가공.merge(일반_특약정보_가공, on='RRNR_TTY_CD', how='left')
일반_특약보종별_직전1년경과보험료_가공.drop(['RRNR_TTY_CD', 'ARC_INPL_CD', 'DMFR_DVCD'], axis=1, inplace=True)

## 손해액
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액.copy()
일반_특약보종별_직전1년손해액_가공['DMFR_DVCD'] = '01'
일반_특약보종별_직전1년손해액_가공['BOZ_CD'] = clsf_boz_cd(일반_특약보종별_직전1년손해액_가공, 일반_상품정보)
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액_가공.merge(일반_특약정보_가공, on='RRNR_TTY_CD', how='left')
일반_특약보종별_직전1년손해액_가공.drop(['RRNR_TTY_CD', 'ARC_INPL_CD', 'DMFR_DVCD'], axis=1, inplace=True)

In [167]:
def get_ret_risk_rate_by_risk_coef(comm: pd.DataFrame, prem: pd.DataFrame, loss_ratio: float, risk_coef: float) -> pd.DataFrame:
    comm2 = comm \
        .assign(
            COMM_RATE_BASE = lambda x: np.fmax(np.fmin(x['CMSN_MULT_RT']*(x['BSE_LSRT']-loss_ratio)+x['CMSN_ADD_RT'], x['TOP_CMSN_RT']), x['LWT_CMSN_RT']),
            COMM_RATE_SHOCKED = lambda x: np.fmax(np.fmin(x['CMSN_MULT_RT']*(x['BSE_LSRT']-(1+risk_coef))+x['CMSN_ADD_RT'], x['TOP_CMSN_RT']), x['LWT_CMSN_RT']),
        ) \
        [['TTY_YR', 'COMM_RATE_BASE', 'COMM_RATE_SHOCKED']]
    prem2 = prem.merge(comm2, on='TTY_YR', how='left') \
        .assign(
            ORI_EXP_LOSS_BASE = lambda x: x['ELP_PRM']*loss_ratio,
            ORI_EXP_LOSS_SHOCKED = lambda x: x['ELP_PRM']*(1+risk_coef),
            ORI_EXP_LOSS_DIFF = lambda x: x['ORI_EXP_LOSS_SHOCKED'] - x['ORI_EXP_LOSS_BASE'],
            RET_EXP_LOSS_BASE = lambda x: (x['ELP_PRM']-x['T02_RN_ELP_PRM'])*loss_ratio,
            RET_EXP_LOSS_SHOCKED = lambda x: (x['ELP_PRM']-x['T02_RN_ELP_PRM'])*(1+risk_coef),
            RET_EXP_LOSS_DIFF = lambda x: x['RET_EXP_LOSS_SHOCKED'] - x['RET_EXP_LOSS_BASE'],
            COMM_BASE = lambda x: x['T02_RN_ELP_PRM']*x['COMM_RATE_BASE'],
            COMM_SHOCKED = lambda x: x['T02_RN_ELP_PRM']*x['COMM_RATE_SHOCKED'],
            COMM_DIFF = lambda x: x['COMM_SHOCKED'] - x['COMM_BASE'],
        ) \
        .drop(['COMM_RATE_BASE', 'COMM_RATE_SHOCKED'], axis=1) \
        .astype({'T02_RN_ELP_PRM': float}) \
        .sort_values(by=['BOZ_CD', 'TTY_YR'])
    return prem2


# 예시
# 근재보험특약, 기술보험특약, 재물보험특약, 배상책임보험특약, 해외PST
특약명 = "기술보험특약"
prem = 일반_특약보종별_직전1년경과보험료_가공.query('TTY_CD_GRP == @특약명').groupby(['TTY_YR', 'BOZ_CD'], as_index=False)[['ELP_PRM', 'T02_RN_ELP_PRM']].sum()
comm = 일반_특약수수료_가공.query('TTY_CD_GRP == @특약명').drop('TTY_CD_GRP', axis=1)
loss_ratio = 0.6
risk_coef = 0.55
get_ret_risk_rate_by_risk_coef(comm, prem, loss_ratio, risk_coef)

,TTY_YR,BOZ_CD,ELP_PRM,T02_RN_ELP_PRM,ORI_EXP_LOSS_BASE,ORI_EXP_LOSS_SHOCKED,ORI_EXP_LOSS_DIFF,RET_EXP_LOSS_BASE,RET_EXP_LOSS_SHOCKED,RET_EXP_LOSS_DIFF,COMM_BASE,COMM_SHOCKED,COMM_DIFF
0,2013,A002,"7,545,087","5,607,018","4,527,052","11,694,885","7,167,833","1,162,841","3,004,007","1,841,166","1,996,098","1,794,246","-201,853"
1,2014,A002,"217,202,262","123,040,504","130,321,357","336,663,506","206,342,149","56,497,055","145,950,725","89,453,670","43,802,419","39,372,961","-4,429,458"
2,2015,A002,"37,594,020","15,393,040","22,556,412","58,270,731","35,714,319","13,320,588","34,411,519","21,090,931","5,479,922","4,925,773","-554,149"
3,2016,A002,"207,109,805","50,539,226","124,265,883","321,020,198","196,754,315","93,942,347","242,684,397","148,742,050","17,890,886","16,172,552","-1,718,334"
5,2017,A002,"718,704,908","131,154,554","431,222,945","1,113,992,607","682,769,663","352,530,212","910,703,049","558,172,836","46,428,712","41,969,457","-4,459,255"
7,2018,A002,"2,149,270,575","541,190,673","1,289,562,345","3,331,369,391","2,041,807,046","964,847,941","2,492,523,848","1,527,675,907","188,875,545","173,181,015","-15,694,530"
9,2019,A002,"2,520,270,388","817,291,327","1,512,162,233","3,906,419,101","2,394,256,869","1,021,787,437","2,639,617,545","1,617,830,108","281,148,216","261,533,225","-19,614,992"
4,2016,A003,"2,066,853","1,446,796","1,240,112","3,203,622","1,963,510","372,034","961,088","589,054","512,166","462,975","-49,191"
6,2017,A003,"15,817,764","11,072,435","9,490,658","24,517,534","15,026,876","2,847,197","7,355,260","4,508,063","3,919,642","3,543,179","-376,463"
8,2018,A003,"1,608,908,182","1,113,899,241","965,344,909","2,493,807,682","1,528,462,773","297,005,365","767,263,859","470,258,494","388,750,835","356,447,757","-32,303,078"


In [ ]:
def get_ret_risk_rate_by_loss_dist(comm: pd.DataFrame, prem: pd.DataFrame, loss: pd.DataFrame, loss_ratio: float, cv: pd.DataFrame) -> pd.DataFrame:
    """특약별 보유리스크율(손해율분포법)"""